In [223]:
%run ../utils/configuration.ipynb

In [224]:
%run ../03_Models/0300_common_functions.ipynb

In [225]:
import polars as pl

# 1 - Reading Dataset and  Merge files

In [226]:
def readAndMergeFiles():
    try:
        df_demographics = pl.read_excel(f"{raw_path}/Telco_customer_churn_demographics.xlsx")
        df_location = pl.read_excel(f"{raw_path}/Telco_customer_churn_location.xlsx")
        df_population = pl.read_excel(f"{raw_path}/Telco_customer_churn_population.xlsx")
        df_services = pl.read_excel(f"{raw_path}/Telco_customer_churn_services.xlsx")
        df_status = pl.read_excel(f"{raw_path}/Telco_customer_churn_status.xlsx")

        #Rename columns to snake_case
        df_demographics = df_demographics.rename({c: to_snake_case(c) for c in df_demographics.columns})
        df_location = df_location.rename({c: to_snake_case(c) for c in df_location.columns})
        df_population = df_population.rename({c: to_snake_case(c) for c in df_population.columns})
        df_services = df_services.rename({c: to_snake_case(c) for c in df_services.columns})
        df_status = df_status.rename({c: to_snake_case(c) for c in df_status.columns})

        #Drop duplicate or unnecessary columns
        df_demographics = df_demographics.drop(['count'])
        df_location = df_location.drop(["count", "country", "state"])

        df_population = df_population.select([pl.col('zip_code'), pl.col('population')])
        
        #Merge dataframes
        df_merge = df_services.join(df_demographics, on="customer_id", how="left")\
                        .join(df_location, on="customer_id", how="left")\
                        .join(df_status, on="customer_id", how="left")\
                        .join(df_population, on="zip_code", how="left")
        return df_merge
    except Exception as e:
        print(f"Error: {e}")
        sys.exit(1)

In [227]:
def setIndex(df_pandas):
    df_pandas.set_index('customer_id', inplace = True)
    return df_pandas

In [228]:
def splitMergedData(df):
    df_train = df.sample(
        fraction=0.8,
        with_replacement=False,
        shuffle=True
    )

    df_test = df.join(df_train, on="customer_id", how="anti")
    return df_train, df_test

# 2 - Pre Processing and Feature Engineering

In [229]:
def solvingOutliers(df):
    q1, q3 = df.select([
        pl.col("number_of_referrals").quantile(0.25).alias("q1"),
        pl.col("number_of_referrals").quantile(0.75).alias("q3")
    ]).row(0)

    upper_bound = q3 + (q3 - q1) * 1.5

    df = df.with_columns(
        pl.col("number_of_referrals")
        .clip(upper_bound=upper_bound)
    )

    q1, q3 = df.select([
        pl.col("avg_monthly_gb_download").quantile(0.25).alias("q1"),
        pl.col("avg_monthly_gb_download").quantile(0.75).alias("q3")
    ]).row(0)

    upper_bound = q3 + (q3 - q1) * 1.5

    df = df.with_columns(
        pl.col("avg_monthly_gb_download")
        .clip(upper_bound=upper_bound)
    )

    q1, q3 = df.select([
        pl.col("total_long_distance_charges").quantile(0.25).alias("q1"),
        pl.col("total_long_distance_charges").quantile(0.75).alias("q3")
    ]).row(0)

    upper_bound = q3 + (q3 - q1) * 1.5

    df = df.with_columns(
        pl.col("total_long_distance_charges")
        .clip(upper_bound=upper_bound)
    )

    q1, q3 = df.select([
        pl.col("total_revenue").quantile(0.25).alias("q1"),
        pl.col("total_revenue").quantile(0.75).alias("q3")
    ]).row(0)

    upper_bound = q3 + (q3 - q1) * 1.5

    df = df.with_columns(
        pl.col("total_revenue")
        .clip(upper_bound=upper_bound)
    )

    q1, q3 = df.select([
        pl.col("satisfaction_score").quantile(0.25).alias("q1"),
        pl.col("satisfaction_score").quantile(0.75).alias("q3")
    ]).row(0)

    lower_bound = q1 - (q3 - q1) * 1.5

    df = df.with_columns(
        pl.when(pl.col("satisfaction_score") < lower_bound)
        .then(pl.lit(lower_bound))
        .otherwise(pl.col("satisfaction_score"))
        .alias("satisfaction_score")
    )

    return df

In [230]:
def applyPowerTransformations(df):
    cols = [
        'avg_monthly_long_distance_charges',
        'avg_monthly_gb_download',
        'monthly_charge',
        'total_charges',
        'total_refunds',
        'total_extra_data_charges',
        'total_revenue',
        'population'
    ]
    for col in cols:
        df = df.with_columns(
            pl.col(col).cbrt().alias(col)
        )
    return df


In [231]:
valid_vals = ["yes", "no", "male", "female"]

bivalue_columns = [
    'referred_a_friend'
    ,'phone_service'
    ,'multiple_lines'
    ,'internet_service'
    ,'online_security'
    ,'online_backup'
    ,'device_protection_plan'
    ,'premium_tech_support'
    ,'streaming_tv'
    ,'streaming_movies'
    ,'streaming_music'
    ,'unlimited_data'
    ,'paperless_billing'
    ,'gender'
    ,'churn_label'
    ,'under_30'	
    ,'senior_citizen'	
    ,'married'	
    ,'dependents'
]

def encodeBivariateVariables(df, bivalue_columns, valid_vals):
    for col in bivalue_columns:
        df = df.with_columns(
            pl.col(col).str.to_lowercase().str.strip_chars()
        )

        mode_val = (
            df
            .filter(pl.col(col).is_in(valid_vals))
            .select(pl.col(col).mode())
            .item()
        )

        df = df.with_columns(
            pl.when(pl.col(col).is_in(valid_vals))
                .then(pl.col(col))
                .otherwise(pl.lit(mode_val))
                .alias(col)
        )

        df = df.with_columns(
            pl.when(pl.col(col).is_in(["yes", "male"]))
                .then(1)
            .when(pl.col(col).is_in(["no", "female"]))
                .then(0)
            .otherwise(None)
            .cast(pl.Int32)
            .alias(col)
        )
    return df

In [232]:
def encodeMuitivariateVariables(df):
    valid_vals = ["Offer A", "Offer B", "Offer C","Offer D", "Offer E"]
    col = "offer"
    mode_val = (
        df
        .filter(pl.col(col).is_in(valid_vals))
        .select(pl.col(col).mode())
        .item()
    )

    df = df.with_columns(
        pl.when(pl.col(col).is_in(valid_vals))
            .then(pl.col(col))
            .otherwise(pl.lit(mode_val))
            .alias(col)
    )

    df = df.with_columns(
        pl.when(pl.col(col) == "Offer A")
            .then(1)
            .when(pl.col(col) == "Offer B")
            .then(2)
            .when(pl.col(col) == "Offer C")
            .then(3)
            .when(pl.col(col) == "Offer D")
            .then(4)
            .when(pl.col(col) == "Offer E")
            .then(5)
            .otherwise(None)
            .cast(pl.Int32)
            .alias(col)
    )

    valid_vals = ["DSL", "Cable", "Fiber Optic"]
    col = "internet_type"
    mode_val = (
        df
        .filter(pl.col(col).is_in(valid_vals))
        .select(pl.col(col).mode())
        .item()
    )

    df = df.with_columns(
        pl.when(pl.col(col).is_in(valid_vals))
            .then(pl.col(col))
            .otherwise(pl.lit(mode_val))
            .alias(col)
    )

    df = df.with_columns(
        pl.when(pl.col(col) == "DSL")
            .then(1)
            .when(pl.col(col) == "Cable")
            .then(2)
            .when(pl.col(col) == "Fiber Optic")
            .then(3)
            .otherwise(None)
            .cast(pl.Int32)
            .alias(col)
    )

    valid_vals = ["Month-to-Month", "One Year", "Two Year"]
    col = "contract"
    mode_val = (
        df
        .filter(pl.col(col).is_in(valid_vals))
        .select(pl.col(col).mode())
        .item()
    )

    df = df.with_columns(
        pl.when(pl.col(col).is_in(valid_vals))
            .then(pl.col(col))
            .otherwise(pl.lit(mode_val))
            .alias(col)
    )

    df = df.with_columns(
        pl.when(pl.col(col) == "Month-to-Month")
            .then(1)
            .when(pl.col(col) == "One Year")
            .then(2)
            .when(pl.col(col) == "Two Year")
            .then(3)
            .otherwise(None)
            .cast(pl.Int32)
            .alias(col)
    )

    return df

In [233]:
def featureEngineering(df):
    valid_vals = ["Churned", "Joined", "Stayed"]
    col = "customer_status"
    mode_val = (
        df
        .filter(pl.col(col).is_in(valid_vals))
        .select(pl.col(col).mode())
        .item()
    )

    df = df.with_columns(
        pl.when(pl.col(col).is_in(valid_vals))
            .then(pl.col(col))
            .otherwise(pl.lit(mode_val))
            .alias(col)
    )

    df = df.with_columns(
        pl.when(pl.col(col) == "Churned")
            .then(0)
            .otherwise(1)
            .cast(pl.Int32)
            .alias(col)
    )

    col = "churn_category"

    df = df.with_columns(
        pl.when((pl.col(col).is_in(["NO_LABEL", None,"", "null"])) | (pl.col(col).is_null()))
            .then(pl.lit("Other"))
            .otherwise(pl.col(col))
            .alias(col)
    )

    col = "total_extra_data_charges"

    df = df.with_columns(
        pl.when(pl.col(col) > 0)
            .then(pl.lit(1))
            .otherwise(pl.col(col))
            .alias(col)
    )

    col = "total_refunds"

    df = df.with_columns(
        pl.when(pl.col(col) > 0)
            .then(pl.lit(1))
            .otherwise(pl.col(col))
            .alias(col)
    )

    col = "number_of_dependents"

    df = df.with_columns(
        pl.when(pl.col(col) > 0)
            .then(pl.lit(1))
            .otherwise(pl.col(col))
            .alias(col)
    )

    df = df.to_dummies(columns=["payment_method"])
    df = df.to_dummies(columns=["churn_category"])

    return df

In [234]:
def dropColumns(df):
    df = df.drop([#'customer_id',
                'under_30', 
                'count', 
                'quarter', 
                'lat_long',	
                'latitude',	
                'longitude', 
                'married', 
                'total_long_distance_charges', 
                'tenure_in_months', 
                'city', 
                'churn_reason',
                'senior_citizen',
                'referred_a_friend',
                'customer_status',
                'zip_code',
                'churn_label',
                'churn_category_Other',
                'churn_category_Competitor',
                'churn_category_Dissatisfaction',
                'churn_category_Attitude',
                'churn_category_Price',
                'quarter_right',
                'count_right'])
    return df

In [235]:
def convertToPandas(df):
    return df.to_pandas()

In [236]:
def setIndex(df):
    df.set_index('customer_id', inplace = True)
    return df

In [237]:
import os
os.environ["LOKY_MAX_CPU_COUNT"] = "8"
from imblearn.over_sampling import SMOTE

def applySMOTE(df):
    X = df.drop("churn_value", axis=1)   
    y = df["churn_value"]                

    smote = SMOTE(random_state=42)

    X_res, y_res = smote.fit_resample(X, y)

    df_balanced = pd.concat([X_res, y_res], axis=1)

    #df = pl.DataFrame(df_balanced)
    return df_balanced

# Predition

In [238]:
def predict(df_test, df, model):
    X_train = df.drop(['churn_value'], axis=1)
    y_train = df['churn_value'].copy()

    X_test = df_test.drop(['churn_value'], axis=1)

    X_train = apply_scale(X_train)

    final_model = model.fit(X_train, y_train)

    X_test = apply_scale(X_test)

    predict_proba_test = final_model.predict_proba(X_test)
    return predict_proba_test

In [252]:
import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

def confusionMatrixBasedOnPredictedProb(predict_proba, df_test, treshold=0.420183):
    final_pred = []

    for value in predict_proba[:,1]:
        if (value>=treshold):
            final_pred.append(1)
        else:
            final_pred.append(0)

    len([x for x in final_pred if x==0])

    answer = pd.DataFrame()
    answer['customer_id'] = df_test.index
    answer['churn_value'] = final_pred

    df_test_to_av = df_test 
    df_test_to_av['customer_id'] = df_test.index

    df_from_model = pl.DataFrame(answer).select(pl.col("customer_id"), pl.col("churn_value").alias("churn_value_predicted"))
    df_test_to_av = pl.DataFrame(df_test_to_av).select(pl.col("customer_id"), pl.col("churn_value").alias("churn_value"))

    both_df = df_from_model.join(df_test_to_av, on="customer_id") 

    confusion = (
        both_df
        .with_columns([
            ((pl.col("churn_value_predicted") == 1) & (pl.col("churn_value") == 1))
                .cast(pl.Int64)
                .alias("TP"),

            ((pl.col("churn_value_predicted") == 1) & (pl.col("churn_value") == 0))
                .cast(pl.Int64)
                .alias("FP"),

            ((pl.col("churn_value_predicted") == 0) & (pl.col("churn_value") == 0))
                .cast(pl.Int64)
                .alias("TN"),

            ((pl.col("churn_value_predicted") == 0) & (pl.col("churn_value") == 1))
                .cast(pl.Int64)
                .alias("FN"),
        ])
        .select([
            pl.sum("TP").alias("True Positives"),
            pl.sum("FP").alias("False Positives"),
            pl.sum("TN").alias("True Negatives"),
            pl.sum("FN").alias("False Negatives"),
        ])
    )

    return confusion

# Train

In [240]:
env = 'dev'
df = readAndMergeFiles()
if env == 'dev':
    df_train, df_test = splitMergedData(df)
    df = df_train
df = solvingOutliers(df)
df = applyPowerTransformations(df)
df = encodeBivariateVariables(df, bivalue_columns, valid_vals)
df = encodeMuitivariateVariables(df)
df = featureEngineering(df)
df = dropColumns(df)
df = convertToPandas(df)
df = setIndex(df)
df = applySMOTE(df)

## 3 - Definição das perspectivas

In [241]:
df_1 = apply_feature_selection(df, 1)
df_2 = apply_feature_selection(df, 2)
df_3 = apply_feature_selection(df, 3)

## 4 -Definição dos modelos

In [242]:
model_NN_1 = MLPClassifier(activation = 'relu',
                               hidden_layer_sizes = (320),
                               learning_rate = 'invscaling',
                               learning_rate_init = 0.01,
                               solver = 'adam', max_iter = 2000, random_state = 99)

In [243]:
model_NN_2 = MLPClassifier(activation = 'relu',
                               hidden_layer_sizes = (280),
                               learning_rate = 'constant',
                               learning_rate_init = 0.01,
                               solver = 'sgd', max_iter = 2000, random_state = 99)

In [244]:
model_NN_3 =  MLPClassifier(activation = 'relu',
                               hidden_layer_sizes = (290),
                               learning_rate = 'constant',
                               learning_rate_init = 0.0001,
                               solver = 'adam', max_iter = 2000, random_state = 99)

# Test

In [245]:
df_test = solvingOutliers(df_test)
df_test = applyPowerTransformations(df_test)
df_test = encodeBivariateVariables(df_test, bivalue_columns, valid_vals)
df_test = encodeMuitivariateVariables(df_test)
df_test = featureEngineering(df_test)
df_test = dropColumns(df_test)
df_test = convertToPandas(df_test)
df_test = setIndex(df_test)

In [246]:
df_test_1 = apply_feature_selection(df_test, 1)
df_test_2 = apply_feature_selection(df_test, 2)
df_test_3 = apply_feature_selection(df_test, 3)

In [247]:
predict_proba_1 = predict(df_test_1, df_1, model_NN_1)
predict_proba_2 = predict(df_test_2, df_2, model_NN_2)
predict_proba_3 = predict(df_test_3, df_3, model_NN_3)

In [248]:
confusion_1 = confusionMatrixBasedOnPredictedProb(predict_proba_1, df_test_1, 0.326077)
confusion_2 = confusionMatrixBasedOnPredictedProb(predict_proba_2, df_test_2, 0.420183)
confusion_3 = confusionMatrixBasedOnPredictedProb(predict_proba_3, df_test_3, 0.355473)

C:\Users\Danie\AppData\Local\Temp\ipykernel_29304\2798319307.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_to_av['customer_id'] = df_test.index
C:\Users\Danie\AppData\Local\Temp\ipykernel_29304\2798319307.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_to_av['customer_id'] = df_test.index
C:\Users\Danie\AppData\Local\Temp\ipykernel_29304\2798319307.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [249]:
confusion_1

True Positives,False Positives,True Negatives,False Negatives
i64,i64,i64,i64
342,307,710,50


In [250]:
confusion_2

True Positives,False Positives,True Negatives,False Negatives
i64,i64,i64,i64
266,173,844,126


In [251]:
confusion_3

True Positives,False Positives,True Negatives,False Negatives
i64,i64,i64,i64
295,187,830,97
